# 01 - Pré-processamento

Pipeline padronizado de preparação dos dados.

In [2]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer


In [3]:
df = pd.read_csv('../data/tratados/amostra_19-21.csv', sep= ',', encoding="ISO-8859-1")

In [4]:

X = df.drop('gravidade', axis=1)
y = df['gravidade']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


numeric_features = ['frota']
categorical_features = ['br',
    'uf', 'km', 'municipio', 'dia_semana', 'fase_dia',
    'sentido_via', 'condicao_metereologica', 'tipo_pista',
    'tracado_via', 'uso_solo', 'tipo_veiculo'
]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [4]:
# Salvar o pipeline de pré-processamento
joblib.dump(preprocess, 'preprocess.pkl')

# (opcional, mas recomendado) salvar os splits
joblib.dump(
    (X_train, X_test, y_train, y_test),
    'data_splits.pkl'
)

print("Pré-processamento e splits salvos com sucesso.")


Pré-processamento e splits salvos com sucesso.


In [5]:
X_train

,uf,br,km,municipio,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,data_inversa,ano,frota
44026,GO,40.0,"16,2",LUZIANIA,sÃ¡bado,Plena Noite,Crescente,CÃ©u Claro,Dupla,Reta,Sim,AutomÃ³vel,2019-08-31,2019,66560.0
9134,PE,101.0,"80,6",JABOATAO DOS GUARARAPES,terÃ§a-feira,Pleno dia,Crescente,CÃ©u Claro,MÃºltipla,Retorno Regulamentado,NÃ£o,Semireboque,2021-10-19,2021,217949.0
54794,PR,476.0,135,CURITIBA,quarta-feira,Pleno dia,Decrescente,CÃ©u Claro,MÃºltipla,Reta,Sim,AutomÃ³vel,2020-04-15,2020,1622891.0
106025,SC,101.0,"310,9",LAGUNA,quarta-feira,Plena Noite,Crescente,Nublado,Simples,RotatÃ³ria,NÃ£o,AutomÃ³vel,2021-05-19,2021,31357.0
52102,MS,163.0,"490,2",CAMPO GRANDE,quinta-feira,Plena Noite,Crescente,CÃ©u Claro,Simples,RotatÃ³ria,Sim,Ãnibus,2019-03-21,2019,613060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112865,AL,101.0,"149,5",JEQUIA DA PRAIA,domingo,Pleno dia,Decrescente,Garoa/Chuvisco,Dupla,Declive;Ponte,NÃ£o,Semireboque,2020-05-10,2020,1402.0
113652,MA,316.0,"419,4",PERITORO,segunda-feira,Plena Noite,Decrescente,CÃ©u Claro,Simples,Reta,NÃ£o,Motocicleta,2020-03-02,2020,4302.0
82921,SP,116.0,416,JUQUIA,terÃ§a-feira,Pleno dia,Crescente,Chuva,Dupla,Curva,NÃ£o,AutomÃ³vel,2021-12-28,2021,8502.0
57586,CE,116.0,40,HORIZONTE,quarta-feira,Pleno dia,Decrescente,Sol,Dupla,Reta,Sim,Ãnibus,2019-06-26,2019,23201.0
